In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from collections import defaultdict
import keras
from models import TSN
from keras import backend as K

Using TensorFlow backend.


In [2]:
class dataGenerator(keras.utils.Sequence):

    def __init__(self, filepath, batch_size, ffpath, segments = 3, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.filepath = filepath
        self.ffpath = ffpath
        self.segments = segments
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = defaultdict(list)
        Y = list()
        Xflows = defaultdict(list)
        for index,each in enumerate(batch_x):
            infopath = os.path.join(self.ffpath,each,"info.txt")
            imgpath = os.path.join(self.ffpath,each,"frames")
            flowspath = os.path.join(self.ffpath,each,"flows")
            f = open(infopath,"r")
            total_frames = int(f.readlines()[0].strip().split(':')[1])
            f.close()
            idxs = []
            base = total_frames//self.segments
            low = 1
            for _ in range(self.segments):
                high = min(low + base, total_frames)
                idxs.append(np.random.randint(low, high,1)[0])
                low = high + 1 
            frames = self.getFrames(idxs, imgpath)
            flows = self.getFlows(idxs, flowspath)
            for i in range(self.segments):
                Xframes[i].append(frames[i])
                
            for i in range(self.segments):
                Xflows[i].append(flows[i])
            
            Y.append(batch_y[index])
            
        finalX = dict()
        i = 1
        for key in Xframes.keys():
            finalX['input_'+str(i)] = np.array(Xframes[key])
            i += 1
        for key in Xflows.keys():
            finalX['input_'+str(i)] = np.array(Xflows[key])
            i += 1
            
        finalY = {'output':self.one_hot_encode(np.array(Y))}

        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels


    def getFlows(self,idxs, flowspath):

        stack = list()
        for i in idxs:
            f1 = "flow_x_"+str(i)+".jpg"
            f2 = "flow_y_"+str(i)+".jpg"
            grayx = self.readImg(os.path.join(flowspath,f1))
            grayy = self.readImg(os.path.join(flowspath,f2))
            img = np.stack((grayx,grayy),axis = 2)
            img = np.squeeze(img,axis = 3)
            stack.append(img)
            
        return np.array(stack)
    
    def readImg(self,path):
        img = cv2.imread(path)
        grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        grayimg = np.expand_dims(grayimg,axis = 2)
        return grayimg
    
    
    def getFrames(self,idxs, imgpath):

        stack = list()
        for i in idxs:
            framename = "frame_"+str(i)+".jpg"
            stack.append(self.readImg(os.path.join(imgpath,framename)))
            
        return np.array(stack)

In [12]:
class dataGeneratorOntheFly(keras.utils.Sequence):

    def __init__(self, datapath, filepath, batch_size, segments = 3, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.datapath = datapath
        self.filepath = filepath
        self.segments = segments
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = defaultdict(list)
        Y = list()
        Xflows = defaultdict(list)
        for index,each in enumerate(batch_x):
            cap = cv2.VideoCapture(os.path.join(datapath,each))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            idxs = []
            base = total_frames//self.segments
            low = 1
            for _ in range(self.segments):
                high = min(low + base, total_frames)
                idxs.append(np.random.randint(low, high,1)[0])
                low = high + 1
            frames,flows = self.getFramesandFlows(idxs, cap)
            for i in range(self.segments):
                Xframes[i].append(frames[i])
                
            for i in range(self.segments):
                Xflows[i].append(flows[i])
            
            Y.append(batch_y[index])
            
        finalX = dict()
        i = 1
        for key in Xframes.keys():
            finalX['input_'+str(i)] = np.array(Xframes[key])
            i += 1
        for key in Xflows.keys():
            finalX['input_'+str(i)] = np.array(Xflows[key])
            i += 1
            
        finalY = {'output':self.one_hot_encode(np.array(Y))}

        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels


    def getFramesandFlows(self,idxs,cap):
        
        frames = list()
        flows = list()
        for each in idxs:
            
            cap.set(cv2.CAP_PROP_POS_FRAMES,each)
            _,thisFrame = cap.read()
            cap.set(cv2.CAP_PROP_POS_FRAMES,each-1)
            _,prevFrame = cap.read()
            
            thisFrame = cv2.resize(thisFrame,(224,224))
            prevFrame = cv2.resize(prevFrame,(224,224))
            
            thisFrame  = cv2.cvtColor(thisFrame,cv2.COLOR_BGR2GRAY)
            prevFrame  = cv2.cvtColor(prevFrame,cv2.COLOR_BGR2GRAY)
            
            optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
            flow = optical_flow.calc(prevFrame, thisFrame, None)
            flow_x = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flow_y = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
            
            thisFrame = np.expand_dims(thisFrame,axis = 2)
            flow_x = np.expand_dims(flow_x, axis = 2)
            flow_y = np.expand_dims(flow_y, axis = 2)
            
            thisFlow = np.stack((flow_x,flow_y),axis = 2)
            thisFlow = np.squeeze(thisFlow,axis = 3)
            
            frames.append(thisFrame)
            flows.append(thisFlow)
        
        return np.array(frames), np.array(flows)

In [5]:
# folderpath = "../ucfTrainTestlist"
# datapath = "../../UCF-101"
# filename = "trainlist01.txt"
# dg = dataGeneratorNew(datapath,os.path.join(folderpath,filename),batch_size = 128)



folderpath = "../ucfTrainTestlist"
filenameTrain = "custom2Train.txt"
filenameVal = "custom2Val.txt"

filepath = os.path.join(folderpath,filenameTrain)
ffpath = os.path.join("../../VidRecognizer_old/FramesFlows/custom2")
dgTrain = dataGenerator(filepath,16,ffpath)

filepath = os.path.join(folderpath,filenameVal)
ffpath = os.path.join("../../VidRecognizer_old/FramesFlows/custom2")
dgVal = dataGenerator(filepath,16,ffpath)

In [6]:
#Create and Compile model
K.clear_session()
model = TSN()
model.compile(optimizer= keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# model.fit([np.array(framesSegments[0]),np.array(framesSegments[1]),np.array(framesSegments[2]),np.array(flows[0]),np.array(flows[1]),np.array(flows[2])],yFramesTrueSegments[0],batch_size = 3, epochs = 1)
np.random.seed(0)
#Fit generator
history = model.fit_generator(dgTrain,epochs = 15,validation_data = dgVal)

Epoch 1/15
30/30 [==============================] - 161s 5s/step - loss: 8.3029 - acc: 0.0188 - val_loss: 8.9750 - val_acc: 0.0446
Epoch 2/15
30/30 [==============================] - 19s 621ms/step - loss: 10.8320 - acc: 0.0604 - val_loss: 13.2260 - val_acc: 0.0357
Epoch 3/15
30/30 [==============================] - 19s 625ms/step - loss: 11.3603 - acc: 0.0479 - val_loss: 10.4487 - val_acc: 0.0402
Epoch 4/15
30/30 [==============================] - 19s 620ms/step - loss: 9.2420 - acc: 0.0312 - val_loss: 10.1773 - val_acc: 0.0446
Epoch 5/15
30/30 [==============================] - 19s 623ms/step - loss: 8.9759 - acc: 0.0500 - val_loss: 9.9643 - val_acc: 0.0580
Epoch 6/15
30/30 [==============================] - 19s 626ms/step - loss: 9.6794 - acc: 0.0521 - val_loss: 9.0557 - val_acc: 0.0357
Epoch 7/15
30/30 [==============================] - 19s 619ms/step - loss: 8.8694 - acc: 0.0458 - val_loss: 9.5581 - val_acc: 0.0446
Epoch 8/15
30/30 [==============================] - 19s 626ms/step

In [8]:
print(history.history)

{'val_loss': [8.975027867725917, 13.225984437125069, 10.448716708592006, 10.177278774125236, 9.964291095733643, 9.055739266531807, 9.558145454951696, 9.714786870138985, 9.500130517142159, 8.646963851792473, 10.596976348331996, 9.655028581619263, 9.944759028298515, 8.333287826606206, 9.839397941316877], 'val_acc': [0.044642857142857144, 0.03571428571428571, 0.04017857142857143, 0.044642857142857144, 0.05803571428571429, 0.03571428571428571, 0.044642857142857144, 0.03125, 0.05357142857142857, 0.044642857142857144, 0.05803571428571429, 0.06696428571428571, 0.06696428571428571, 0.05803571428571429, 0.09821428571428571], 'loss': [8.30285031000773, 10.83195405403773, 11.360308972994487, 9.241976841290791, 8.975876871744791, 9.679358990987142, 8.869439220428466, 9.353040246168772, 8.739012694358825, 9.283133284250896, 9.669521749019623, 10.791933949788412, 10.359678117434184, 9.57100871404012, 8.651216745376587], 'acc': [0.01875, 0.06041666666666667, 0.04791666666666667, 0.03125, 0.05, 0.0520